In [36]:
import sys
import os
sys.path.append('C:/Users/dinle/Code/Learning/Py_Code/AI_Learning/mnist')
sys.path.append('C:/Users/dinle/Code/Learning/Py_Code/AI_Learning/Node_Layer_Network')
from Node_Layer_Network.Networks.multi_layer_net_extend import MultiLayerNetExtend as MLE1
from Node_Layer_Network.Networks.simple_convnet import SimpleConvNet as CVS1
from Node_Layer_Network.Networks.deep_convnet import DeepConvNet as CVD1
from mnist.common.multi_layer_net_extend import MultiLayerNetExtend as MLE2
from mnist.ch07.simple_convnet import SimpleConvNet as CVS2
from mnist.ch08.deep_convnet import DeepConvNet as CVD2
import matplotlib.pyplot as plt
from mnist.dataset.mnist import load_mnist
from mnist.common.util import smooth_curve
from mnist.common.optimizer import *
import numpy as np
import time


In [37]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)
# x_train = x_train[:1000]
# t_train = t_train[:1000]
train_size = x_train.shape[0]
test_size = x_test.shape[0]
batch_size = 256
max_iterations = 1000
print_iter = 10
mi = np.arange(max_iterations)

In [38]:
def test(test_list, max_acc=None):
    networks = {}
    train_acc = {}
    train_loss = {}
    test_acc = {}
    test_loss = {}
    optimizer = {}
    total_step = {}
    time_record = {}

    for key in test_list.keys():
        # 기록 리스트 생성
        train_acc[key] = []
        train_loss[key] = []
        test_acc[key] = []
        test_loss[key] = []
        total_step[key] = 0
        time_record[key] = 0

        # nSigmoid 경우 파라미터
        # key == 'nSigmoid_8' -> act = nSigmoid, threshold = 8
        networks[key] = test_list[key]['network']

        # optimizer 생성
        opt = test_list[key]['opt']
        lr = test_list[key]['lr']
        if opt == 'SGD':
            optimizer[key] = SGD(lr)
        elif opt == 'Momentum':
            optimizer[key] = Momentum(lr)
        elif opt == 'AdaGrad':
            optimizer[key] = AdaGrad(lr)
        elif opt == 'Adam':
            optimizer[key] = Adam(lr)
        else:
            assert 'no opt' + opt

    # 파라미터 통일
    # networks1 = networks['My']
    # networks2 = networks['Text']
    # for k in networks1.params.keys():
    #     for i in range(networks1.params[k].shape[0]):
    #         networks1.params[k][i] = networks2.params[k][i].copy()
    # networks['My'].params['W1'][0][0] = 100
    # print(networks['My'].params['W1'] - networks['Text'].params['W1'])

    # 학습
    for i in range(max_iterations):
        # if 0 not in total_step.values():
        #     print("All test list training Done\nStep: " + str(i))
        #     break

        # mini-batch train
        batch_mask = np.random.choice(train_size, batch_size)
        x_batch = x_train[batch_mask]
        t_batch = t_train[batch_mask]

        # mini-batch test
        batch_mask_t = np.random.choice(test_size, batch_size)
        x_batch_t = x_test[batch_mask_t]
        t_batch_t = t_test[batch_mask_t]

        # print_iter 회마다 경과 출력
        if i % print_iter == 0:
            print( "="*15 + "iteration:" + str(i) + "="*15)
            print("{:^9}|{:^9}|{:^9}|{:^9}".format('model','time','acc','loss'))

        # 학습 & 추론 & 기록
        for key in test_list.keys():
            if total_step[key] != 0:
                # Max acc 에 도달해 학습이 끝난 test model
                continue
            else:
                start = time.time()
                # CV 모델 데이터 처리
                if isinstance(networks[key], (CVS1, CVS2, CVD1, CVD2)):
                    x_batch = x_batch.reshape(-1, 1, 28, 28)
                    x_batch_t = x_batch_t.reshape(-1, 1, 28, 28)

                # 학습(역전파)
                grads = networks[key].gradient(x_batch, t_batch)
                optimizer[key].update(networks[key].params, grads)

                # 추론(순전파)
                if isinstance(networks[key], (MLE1, CVS1, CVD1)):
                    tr_acc, tr_loss = networks[key].acc_and_loss(x_batch, t_batch)
                    ts_acc, ts_loss = networks[key].acc_and_loss(x_batch_t, t_batch_t)
                else:
                    if isinstance(networks[key], (CVS1, CVS2, CVD1, CVD2)):
                        tr_acc = networks[key].accuracy(x_batch, t_batch, batch_size)
                        ts_acc = networks[key].accuracy(x_batch_t, t_batch_t, batch_size)
                    else:
                        tr_acc = networks[key].accuracy(x_batch, t_batch)
                        ts_acc = networks[key].accuracy(x_batch_t, t_batch_t)
                    tr_loss = networks[key].loss(x_batch, t_batch)
                    ts_loss = networks[key].loss(x_batch_t, t_batch_t)

                # 기록
                train_acc[key].append(tr_acc)
                train_loss[key].append(tr_loss)
                test_acc[key].append(ts_acc)
                test_loss[key].append(ts_loss)
                end = time.time()
                time_record[key] += (end-start)

                # max accuracy 도달 해당 모델 학습 종료
                if max_acc and max_acc <= ts_acc:
                    total_step[key] = i
                    print(key + " training end!\nacc : " + str(ts_acc) + " step: " + str(i))

                # print_iter 회마다 경과 출력
                if i % print_iter == 0:
                    print("{:^9}| {:0<7.3f} | {:0<.5f} | {:0<.5f}".format(key, time_record[key],ts_acc,ts_loss))

    return train_acc, train_loss, test_acc, test_loss, total_step


def plot(label, datas, t_list, *y_lim):
    for key in t_list:
        plt.plot( smooth_curve(datas[key]), markevery=50, label=key)
    plt.xlabel("iterations")
    plt.ylabel(label)
    if y_lim:
        plt.ylim(y_lim)
    plt.legend()
    plt.show()

In [39]:
net1 = MLE2(input_size=784, hidden_size_list=[100, 100, 100, 100], output_size=10,
            use_dropout=True, dropout_ration=0.5,
            activation='relu', weight_init_std='he', use_batchnorm=False)
# net2 = MLE1(input_size=784, hidden_size_list=[100, 100, 100, 100], output_size=10,
#             use_dropout=do, dropout_ration=dr,
#             activation=act, weight_init_std=std, use_batchnorm=bn)
# net1 = CVD1(back_eta=False)
net2 = CVS2()
net3 = CVD2()

In [40]:
test_list = {
             'MLP':
                 {'network':net1,'opt':'AdaGrad','lr':0.01},
             'Smpl CNN':
                 {'network':net2,'opt':'AdaGrad','lr':0.01},
             'Deep CNN':
                 {'network':net3,'opt':'AdaGrad','lr':0.01},
             }


train_acc, train_loss, test_acc, test_loss, total_step = test(test_list, 0.95)

===============iteration:0===============
  model  |  time   |   acc   |  loss   
   MLP   | 0.02100 | 0.27734 | 2.28364
Smpl CNN | 1.81700 | 0.08984 | 2.26428
Deep CNN | 12.8180 | 0.08203 | 5.88892


KeyboardInterrupt: 

In [ ]:
plot('train_acc', train_acc, test_list)

In [ ]:
plot('train_loss', train_loss, test_list)

In [ ]:
print(total_step)

In [ ]:
plt.subplot(1,2,1)

plt.plot( smooth_curve(train_acc['My']), markevery=50, label='My')
plt.xlabel("iterations")
plt.ylabel('train_acc')
plt.legend()

plt.subplot(1,2,2)
plt.plot( smooth_curve(train_acc['Text']), markevery=50, label='Text')
plt.legend()
plt.show()